In [ ]:
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
#from gensim.parsing.porter import PorterStemmer
#from nltk.corpus import stopwords
import numpy as np
import pickle
from sklearn.cluster import KMeans
#from sklearn.metrics import pairwise_distances
import math
import operator
import matplotlib.pyplot as plt
from os import system
%matplotlib inline

In [ ]:
"""
##
#clean raw text
##

#Initialize the porter stemmer and stopwords
#stemmer = PorterStemmer()
with open('../../Misc/stopwords.txt', 'r') as f:
    stops = []
    for line in f:
        stops.append(line.rstrip())
    stops = set(stops)

#clean the data and save to disk
with open('../../Misc/reviews.txt', 'r') as f:
    with open('../../cleaned_reviews.txt', 'w') as cleaned:
        for line in f:
            line = line.replace('...more', '')
            line = line.replace('.', ' ')
            #tokens = [ stemmer.stem(word) for word in line.lower().rstrip().split(' ') if word not in stops ]
            tokens = [ word.strip() for word in line.lower().rstrip().split(' ') if word not in stops ]
            cleaned.write('{}\n'.format(' '.join(tokens)))
"""
print('Comments')

In [ ]:
##
# Code for creating the model. After created, then just load the saved model
##

#Helpful iterator, credit here: https://rare-technologies.com/word2vec-tutorial/
class MySentences(object):
    
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
        for line in open(self.filename):
            yield line.rstrip().split()

#train word2vec on cleaned data, but do so using the memory saving trick from the link above
sentences = MySentences('../../cleaned_reviews.txt')
model = Word2Vec(sentences, size = 300, workers = 4)
model.save('../../Models/w2v')

#Get vectors only from the model and save to disk
wv = model.wv
wv.save('../../Word_Vectors/wv')

#remove model from memory, we only need the wv's
del model

#print('Comments')

In [2]:
#load word vectors from initial word2vec training
wv = KeyedVectors.load('../../Word_Vectors/wv')

In [3]:
#write master matrix to file
with open('../../Pre_Clustering/wvmaster.csv', 'w') as f:
    
    #save word and its vector disk
    for word, data in wv.vocab.items():
        f.write('{} '.format(word))
        f.write('{}\n'.format(' '.join([ str(element) for element in wv[word] ])))
    
#print('Comments')

In [4]:
#Run the retrofit program (runs in a separate subprocess)
print('Exit code: {}'.format(system(
    'python \
    ../../retrofitting/retrofit.py \
    -i ../../Pre_Clustering/wvmaster.csv \
    -l ../../Misc/seeds.txt \
    -n 10 \
    -o ../../Pre_Clustering/retrofitted_dirty.txt'
)))

Exit code: 0


In [5]:
#output of retrofit doesn't have a header which we need for loading into gensim
with open('../../Pre_Clustering/retrofitted_dirty.txt', 'r') as f:
    lines = f.readlines()

with open('../../Pre_Clustering/retrofitted_dirty.txt', 'w') as f:
    
    f.write('{} {}\n'.format(len(wv.vocab), 300))
    
    for line in lines:
        f.write(line)

del lines

In [6]:
#load the retrofitted vectors in as a gensim object
wv = KeyedVectors.load_word2vec_format('../../Pre_Clustering/retrofitted_dirty.txt', binary = False)
wv.save('../../Word_Vectors/retrofitted_wv')

In [7]:
wv = KeyedVectors.load('../../Word_Vectors/retrofitted_wv')

In [8]:
#create the vocab->index, index->vocab dictionary, and indexed word vectors and save all to disk
vocab2index = {}
index2vocab = {}

with open('../../Pre_Clustering/retrofitted_dirty.txt', 'r') as f: 
    
    lines = f.readlines()
    
    with open('../../Pre_Clustering/retrofitted_clean.txt', 'w') as o:
        
        for line in lines[1:]:
            
            #get the word and its vector separately
            splits = line.rstrip().split(' ')
            word = splits[0]
            vector = splits[1:]
            
            #build the vocab dictionaries
            vocab2index[word] = wv.vocab[word].index
            index2vocab[ vocab2index[word] ] = word
            
            #save the indexed vectors to file for loading later
            o.write('{} '.format(vocab2index[word]))
            o.write('{}\n'.format(' '.join(vector)))
    
    del lines

with open('../../Vector_Tracking/vocab2index.pkl', 'wb') as f:
    pickle.dump(vocab2index, f, pickle.HIGHEST_PROTOCOL)
    
with open('../../Vector_Tracking/index2vocab.pkl', 'wb') as f:
    pickle.dump(index2vocab, f, pickle.HIGHEST_PROTOCOL)

In [12]:
#numk we'll be using
numk = 1000

#vocab -> index
with open('../../Vector_Tracking/vocab2index.pkl', 'rb') as f:
    vocab2index = pickle.load(f)

#index -> vocab
with open('../../Vector_Tracking/index2vocab.pkl', 'rb') as f:
    index2vocab = pickle.load(f)

#master numpy matrix with index as first column and word vector as the rest
X = np.loadtxt('../../Pre_Clustering/retrofitted_clean.txt', delimiter = ' ')

In [13]:
#run k means
kmeans = KMeans(n_clusters = numk, random_state = 0, n_jobs = 4).fit(X[:, 1:])

#save kmeans
with open('../../KMeans/kmeans_{}.pkl'.format(numk), 'wb') as f:
    pickle.dump(kmeans, f, pickle.HIGHEST_PROTOCOL)

#print('Comments')

In [14]:
#load kmeans
with open('../../KMeans/kmeans_{}.pkl'.format(numk), 'rb') as f:
    kmeans = pickle.load(f)

#attach words to labels
clustered_words = {}
for i, label in enumerate(kmeans.labels_):
    clustered_words[ index2vocab[int(X[i, 0])] ] = label

#group words by their labels
fullbags = [ [] for i in range(numk) ]
for k, v in clustered_words.items():
    fullbags[int(v)].append( (k, wv.vocab[k].count) )

#Sort each cluster and trim to top 20.
wordbags = [ [] for i in range(numk) ]
for i, bag in enumerate(fullbags):
    fullbags[i] = [ item[0] for item in sorted(bag, key = operator.itemgetter(1), reverse = True) ]
    wordbags[i] = fullbags[i][0:20]

with open('../../KMeans/fullbags_{}.pkl'.format(numk), 'wb') as f:
    pickle.dump(fullbags, f, pickle.HIGHEST_PROTOCOL)

#save trimmed clusters so we don't have to do anything above again
with open('../../KMeans/wordbags_{}.pkl'.format(numk), 'wb') as f:
    pickle.dump(wordbags, f, pickle.HIGHEST_PROTOCOL)

    #print('Comments')

In [23]:
numk = 1000

#load the wordbags and fullbags from file
with open('../../KMeans/wordbags_{}.pkl'.format(numk), 'rb') as f:
    wordbags = pickle.load(f)

with open('../../KMeans/fullbags_{}.pkl'.format(numk), 'rb') as f:
    fullbags = pickle.load(f)

In [35]:
#group the cluster ids of those that should
#belong to a respective aspect based upon the
#seed words used
amenities = {}
services = {}
location = {}
price = {}

for i in range(1000):
    amenities[i] = 0
    services[i] = 0
    location[i] = 0
    price[i] = 0

with open('../../Misc/amenities.txt', 'r') as f:
    for line in f:
        for i, bag in enumerate(fullbags):
            if line.rstrip().lower() in bag:
                amenities[i] += 1
                    
with open('../../Misc/services.txt', 'r') as f:
    for line in f:
        for i, bag in enumerate(fullbags):
            if line.rstrip().lower() in bag:
                services[i] += 1
                
with open('../../Misc/price.txt', 'r') as f:
    for line in f:
        for i, bag in enumerate(fullbags):
            if line.rstrip().lower() in bag:
                price[i] += 1
                
with open('../../Misc/location.txt', 'r') as f:
    for line in f:
        for i, bag in enumerate(fullbags):
            if line.rstrip().lower() in bag:
                location[i] += 1

In [36]:
ratios = {}
for i in range(1000):
    
    sm = amenities[i] + services[i] + price[i] + location[i]
    
    if sm == 0:
        sm = 1
    
    ratios[i] = {'services': services[i] / float(sm), 'amenities': amenities[i] / float(sm), 'price': price[i] / float(sm), 'location': location[i] / float(sm)}

In [38]:
for i in range(len(ratios)):
    print('Cluster: {}, Services: {}, Amenities: {}, Price: {}, Location: {}'.format(
        i,
        ratios[i]['services'],
        ratios[i]['amenities'],
        ratios[i]['price'],
        ratios[i]['location']
    ))

Cluster: 0, Services: 0.0, Amenities: 0.0, Price: 0.0, Location: 0.0
Cluster: 1, Services: 0.0, Amenities: 0.0, Price: 0.0, Location: 0.0
Cluster: 2, Services: 0.0, Amenities: 0.0, Price: 0.0, Location: 0.0
Cluster: 3, Services: 0.0, Amenities: 0.0, Price: 0.0, Location: 0.0
Cluster: 4, Services: 0.0, Amenities: 0.0, Price: 0.0, Location: 0.0
Cluster: 5, Services: 0.0, Amenities: 0.0, Price: 0.0, Location: 0.0
Cluster: 6, Services: 0.0, Amenities: 0.0, Price: 0.0, Location: 0.0
Cluster: 7, Services: 0.0, Amenities: 0.0, Price: 0.0, Location: 0.0
Cluster: 8, Services: 0.0, Amenities: 0.0, Price: 0.0, Location: 0.0
Cluster: 9, Services: 0.0, Amenities: 0.0, Price: 0.0, Location: 0.0
Cluster: 10, Services: 0.0, Amenities: 0.0, Price: 0.0, Location: 0.0
Cluster: 11, Services: 0.0, Amenities: 0.0, Price: 0.0, Location: 0.0
Cluster: 12, Services: 0.0, Amenities: 0.0, Price: 0.3333333333333333, Location: 0.6666666666666666
Cluster: 13, Services: 0.0, Amenities: 0.0, Price: 0.0, Location: 0.0


// Below was used for seed word creation //

In [ ]:
with open('../../Thesaurus/thesaurus.pkl', 'rb') as f:
    thesaurus = pickle.load(f)

In [ ]:
synSet = set()
for k in thesaurus:
    synSet.add(k.lower())
    
vocabSet = set()
for k in wv.vocab:
    vocabSet.add(k.lower())
    
intSet = synSet & vocabSet

In [ ]:
thesausets = [ set(ws) for ws in fullbags ]
for i in range(len(thesausets)):
    thesausets[i] = thesausets[i] & intSet